In [9]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import cvui
from tqdm.notebook import tqdm 
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np

In [ ]:
def getFramesFromVideo(file_path):
    cap=cv2.VideoCapture(file_path)
    for i in range(int(total)):
        _,frame=cap.read()
        cv2.imshow('board', board)
        key=cv2.waitKey(1) & 0xff;
        if key==27:break
        end_point=(int((i/total)*380),50)   
        bar = cv2.rectangle(bar, start_point, end_point, (0,255,0), -1) 
        cv2.imwrite(subDir+str(i)+'.png',frame)
        cvui.update()
    cv2.destroyAllWindows(); 

In [14]:
##for presentation

b=[98,131,131,255,0,255]
g=[50,115,131,255,8,252]
r = [142,180,99,255,8,252];r_=[0,4,99,255,8,252]

# dp , minDist,param1,param2,minRadius,maxRadius,BlurSize
bC=[1,101,68,8,0,68,1]
gC=[1,66,52,10,0,300,1]
rC=[1,80,52,10,0,700,3]


def getHSV(frame,l,m=[]):
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    LB=np.array(l[::2]);UB=np.array(l[1::2])
    if m!=[]: LB_=np.array(m[::2]);UB_=np.array(m[1::2]);mask=cv2.inRange(hsv,LB,UB) + cv2.inRange(hsv,LB_,UB_)
    else : mask=cv2.inRange(hsv,LB,UB)
    res=cv2.bitwise_and(frame,frame,mask=mask)
    #return res#,cv2.bitwise_not(mask)
    return mask
#dp=1,minDist=512,param1=100,param2=100,minRadius=0,maxRadius=0,blurSize=3
def getHoughCircles(frame,l):
    if l[6]%2 == 0 : l[6] +=1
    frame=cv2.blur(frame,(l[6],l[6]))
    highlight=frame.copy()
    
    if len(frame.shape)==3 : frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    else : highlight=cv2.cvtColor(frame,cv2.COLOR_GRAY2BGR)
    #frame=cv2.blur(frame,(31, 31))
    circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT,  l[0], l[1], param1 = l[2], param2 = l[3], minRadius=l[4], maxRadius=l[5])
    return circles
    
def drawCircles(frame,circlesList,textList=[]) :
    
    font = cv2.FONT_HERSHEY_SIMPLEX ;fontScale = 0.5 ;color = (255, 255, 255) ;thickness = 1
    highlight=frame.copy()

    for (circles,text) in zip(circlesList,textList):
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                highlight=cv2.circle(highlight, (x, y), r, (0, 255, 0), 4)
                highlight=cv2.rectangle(highlight, (x - 3, y - 3), (x + 3, y + 3), (0, 128, 255), -1)
                highlight=cv2.putText(highlight, text, (x,y), font, fontScale, color, thickness, cv2.LINE_AA) 

    return highlight

    
cv2.namedWindow('blur',cv2.WINDOW_NORMAL);
cv2.namedWindow('red',cv2.WINDOW_NORMAL);
cv2.namedWindow('blue',cv2.WINDOW_NORMAL);
cv2.namedWindow('green',cv2.WINDOW_NORMAL);

for i in range ( 7000,10000,5) :
    img = cv2.imread("images/"+str(i)+".png")
    cv2.waitKey(1)
    redCircles=getHoughCircles(getHSV(img,r,r_),rC)
    blueCircles =getHoughCircles(getHSV(img,b),bC)
    greenCircles = getHoughCircles(getHSV(img,g),gC)
    
    cv2.imshow('blur',img)
    cv2.imshow('red',drawCircles(img,[redCircles,blueCircles,greenCircles],["Red ball","Blue ball","Green Ball"]))
    #cv2.imshow('blue',drawCircles(img,blueCircles,"Blue ball"))
    #cv2.imshow('green',drawCircles(img,greenCircles,"Green ball"))
    key= cv2.waitKey(1) & 0xff
    if key==27:break
cv2.destroyAllWindows()

array([[[132.5, 276.5, 112. ],
        [200.5, 208.5,  11.8]]], dtype=float32)

In [46]:
from skimage import io, color, measure, draw, img_as_bool
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt


image = img_as_bool(color.rgb2gray(io.imread('reed.png')))
regions = measure.regionprops(image.astype(int))
bubble = regions[0]

y0, x0 = bubble.centroid
r = bubble.major_axis_length / 2.

def cost(params):
    x0, y0, r = params
    coords = draw.circle(y0, x0, r, shape=image.shape)
    template = np.zeros_like(image)
    template[coords] = 1
    return -np.sum(template == image)

x0, y0, r = optimize.fmin(cost, (x0, y0, r))

import matplotlib.pyplot as plt

f, ax = plt.subplots()
circle = plt.Circle((x0, y0), r)
ax.imshow(image, cmap='gray', interpolation='nearest')
ax.add_artist(circle)
plt.show()

Optimization terminated successfully.
         Current function value: -346222.000000
         Iterations: 40
         Function evaluations: 121


array([0., 0., 0., 0., 0.])

In [89]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np
import cvui
import tkinter as tk
from tkinter.filedialog import askopenfilename 
from tkinter import filedialog
import pickle
from skimage.measure import label, regionprops
import math

var={       'filterFlag':            [True],
            'filterType':            ['hsvFilter'],
            'threshValue':           [0],
            'medianValue':           [0],
            
            'cannyLowValue':         [0],
            'cannyHighValue':        [500],
            
            'hsvLowHue':             [0],
            'hsvHighHue':            [179],
            'hsvLowSaturation':      [0],
            'hsvHighSaturation':     [255],
            'hsvLowValue':           [0],
            'hsvHighValue':          [255],
            
            'houghCirclesFlag':      [False],
            'houghCirclesDP':        [1],
            'houghCirclesPARAM1':    [1],
            'houghCirclesPARAM2':    [1],
            'houghCirclesMINRADIUS': [1],
            'houghCirclesMAXRADIUS': [1],
            'houghCirclesMINDIST':   [1],
            'houghCirclesBLURSIZE' : [3] ,
             
             'lineOrientationFlag':  [False],
             'roi':                  [],
                        
           
            
            'loadPickleFlag':        [False],
            'savePickleFlag':        [False],

            "analysisFlag"  :        ["ballFlag"],
                      
            'subDir':                ['images/'],
            

    }

frameDic = { 'prevFrameNumber':       [0],
            'frameNumber':           [0],
            'startFrame':            [0],
            'endFrame':              [500],
            'totalFrame':            [0],
            'frameShape':            [600,600],
        }

%matplotlib qt

#GUI elements
def frameSliderGUI(board,dic):
    cvui.window(board,10,140,400,70,'frame Selection')
    cvui.text(board,20,180,'FRAME#');
    #if dic['frameNumber'] != dic['prevFrameNumber']:
    cvui.trackbar(board,80,160,330,dic['frameNumber'],dic['startFrame'][0],dic['endFrame'][0],1,'%.0Lf',cvui.TRACKBAR_DISCRETE)    
def filterGroupGUI(board):
    cvui.window(board,10,220,400,720,'Filter Parameters')  #MatrixName,xPos,yPos,width,height,Name
    options=cvui.TRACKBAR_DISCRETE
    cvui.window(board,30,250,360,180,'Filters & Channels')

    #Channels group
    
    
    
    #Filters group
    if cvui.button(board,40,350,85,60,'threshold'):var['filterType']=['threshFilter']
    if cvui.button(board,125,350,85,60,'Canny '):var['filterType']=['cannyFilter']
    if cvui.button(board,210,350,85,60,'HSV'):var['filterType']=['hsvFilter']
    if cvui.button(board,295,350,85,60,'Hough Circles'):var['houghCirclesFlag']=[True]
        
    #Thresh Window
    if var['filterType']==['threshFilter']:
        cvui.window(board,30,440,360,200,'threshold filter')
        cvui.text(board,40,500,'thresh');cvui.trackbar(board,100,480,270,var['threshValue'],0,255,1,'%.0Lf',options)
        cvui.text(board,40,560,'Median');cvui.trackbar(board,100,540,270,var['medianValue'],1,100,1,'%.0Lf',options)
    
    #Canny Window
    elif var['filterType']==['cannyFilter']:
        cvui.window(board,30,440,360,200,'Canny filter')
        cvui.text(board,40,500,'low can');cvui.trackbar(board,100,480,270,var['cannyHighValue'],1,500,1,'%.0Lf',options)
        cvui.text(board,40,560,'high can');cvui.trackbar(board,100,540,270,var['cannyLowValue'],1,500,1,'%.0Lf',options)
    
    #HSV Window
    elif var['filterType']==['hsvFilter']:
        cvui.window(board,30,440,360,400,'HSV filter')
        cvui.text(board,40,500,'low hue');cvui.trackbar(board,100,480,270,var['hsvLowHue'],0,179,1,'%.0Lf',options )
        cvui.text(board,40,560,'high hue');cvui.trackbar(board,100,540,270,var['hsvHighHue'],0,179,1,'%.0Lf',options)
        cvui.text(board,40,620,'low sat');cvui.trackbar(board,100,600,270,var['hsvLowSaturation'],0,255,1,'%.0Lf',options)
        cvui.text(board,40,680,'high sat');cvui.trackbar(board,100,660,270,var['hsvHighSaturation'],0,255,1,'%.0Lf',options)
        cvui.text(board,40,750,'low val');cvui.trackbar(board,100,730,270,var['hsvLowValue'],0,255,1,'%.0Lf',options)
        cvui.text(board,40,810,'high val');cvui.trackbar(board,100,790,270,var['hsvHighValue'],0,255,1,'%.0Lf',options)                     
    
    if cvui.button(board,420+30 , 20,350,60,"Line Orientation"):var["lineOrientationFlag"] = [not var["lineOrientationFlag"][0]] 
    if cvui.button(board,420+30 , 380,350,60,"hough circles"):var["houghCirclesFlag"] = [not var["houghCirclesFlag"][0]] 
    
    if var['houghCirclesFlag']==[True]:
        cvui.window(board,420+30,440,360,460,'hough circles')
        cvui.text(board,420+40,500,'dp');cvui.trackbar(board,420+100,480,270,var['houghCirclesDP'],1,10,1,'%.0Lf',options )
        cvui.text(board,420+40,560,'minDist');cvui.trackbar(board,420+100,540,270,var['houghCirclesMINDIST'],1,179,1,'%.0Lf',options)
        cvui.text(board,420+40,620,'param1');cvui.trackbar(board,420+100,600,270,var['houghCirclesPARAM1'],1,255,1,'%.0Lf',options)
        cvui.text(board,420+40,680,'param2');cvui.trackbar(board,420+100,660,270,var['houghCirclesPARAM2'],1,255,1,'%.0Lf',options)
        cvui.text(board,420+40,750,'minRadius');cvui.trackbar(board,420+100,730,270,var['houghCirclesMINRADIUS'],0,512,1,'%.0Lf',options)
        cvui.text(board,420+40,810,'maxRadius');cvui.trackbar(board,420+100,790,270,var['houghCirclesMAXRADIUS'],0,1000,1,'%.0Lf',options) 
        cvui.text(board,420+40,870,'Blur size');cvui.trackbar(board,420+100,790+60,270,var['houghCirclesBLURSIZE'],1,255,2,'%.0Lf',options) 

        
    
    
    
#     if cvui.button(board,40,290,113,60,'Balls'):var['analysisFlag']=['ballFlag']
#     if cvui.button(board,153,290,113,60,'Wall'):var['analysisFlag']=['wallFlag']
#     if cvui.button(board,266,290,113,60,'Entrance'):var['analysisFlag']=['entranceFlag']
        
    

circlesHist=np.zeros((),np.uint8)    
def getVideoProp(cap):
    frameCount=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fpsCount=int(cap.get(cv2.CAP_PROP_FPS))
    frameHeight=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frameWidth=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    return {'frameCount':frameCount,'fpsCount':fpsCount,'frameHeight':frameHeight,'frameWidth':frameWidth}   
def statusTextGUI(board,cap):
    props=getVideoProp(cap)
    frameCount=props['frameCount']
    fpsCount=props['fpsCount']
    frameHeight=props['frameHeight']
    frameWidth=props['frameWidth']
    cvui.window(board,10,10,400,120,'CAPSTONE TEAM H-FOR TESTING PURPOSES')
    cvui.text(board,50,50,'FPS:' + str(fpsCount));
    cvui.text(board,50,70,'FRAME COUNT:' +str(frameCount));
    cvui.text(board,50,90,'WxH:' + str(frameWidth)+'x' + str(frameHeight));
    if cvui.button(board,250,50,113,30,'load parameters'):var['loadPickleFlag']=[True]
    if cvui.button(board,250,90,113,30,'save parameters'):var['savePickleFlag']=[True]
        
def writePkl(fileName,file):
    with open(fileName, 'wb') as output:pickle.dump(file, output, pickle.HIGHEST_PROTOCOL)
def readPkl(fileName):
    with open(fileName, 'rb') as input:return pickle.load(input)    
        
#Get filters
def getCanny(frame,low,high):
    frame=cv2.Canny(frame,low,high)
    return frame,frame
def getThreshMedian(frame,thresh,median=1):
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    ret,frame=cv2.threshold(frame,thresh,255,cv2.THRESH_BINARY);
    #threshImg=cv2.medianBlur(threshImg,median)
    return frame,frame
def getHSV(frame,lh,hh,ls,hs,lv,hv):
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    if lh > hh :
        
        LB=np.array([lh,ls,lv]);UB=np.array([180,hs,hv]);
        LB_=np.array([0,ls,lv]);UB_=np.array([hh,hs,hv]);
        mask=cv2.inRange(hsv,LB,UB) + cv2.inRange(hsv,LB_,UB_)
    else :
        LB=np.array([lh,ls,lv]);UB=np.array([hh,hs,hv]);
        mask=cv2.inRange(hsv,LB,UB)
        
    res=cv2.bitwise_and(frame,frame,mask=mask)
    return res,cv2.bitwise_not(mask)
    #return mask,mask
def getHoughCircles(frame,dp=1,minDist=512,param1=100,param2=100,minRadius=0,maxRadius=0,blurSize=3):
    if blurSize%2 == 0 : blurSize +=1
    frame=cv2.blur(frame,(blurSize,blurSize))
    #highlight=frame.copy()
    
    if len(frame.shape)==3 : frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    else : highlight=cv2.cvtColor(frame,cv2.COLOR_GRAY2BGR)
    #frame=cv2.blur(frame,(31, 31))
    circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT,  dp, minDist, param1 = param1, param2 = param2, minRadius=minRadius, maxRadius=maxRadius)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            highlight=cv2.circle(highlight, (x, y), r, (0, 255, 0), 4)
            highlight=cv2.rectangle(highlight, (x - 3, y - 3), (x + 3, y + 3), (0, 128, 255), -1)
    return circles,highlight

#apply
def getFrame(frameNumber,cap):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameNumber)
    _,frame=cap.read()
    return frame
def getFile():
    global file_path
    root = tk.Tk()
    root.withdraw()
    try: 
        file_path = filedialog.askopenfilename();
        return file_path
    except :
        pass
def applyFilter(frame,dic):
    if dic['filterType']==['threshFilter']:
        frame,_=getThreshMedian(frame,dic['threshValue'][0],dic['medianValue'][0])
        return frame,_
    
    elif dic['filterType']==['cannyFilter']:
        frame,_=getCanny(frame,dic['cannyLowValue'][0],dic['cannyHighValue'][0])
        return frame,_
    
    elif dic['filterType']==['hsvFilter']:
        frame,mask=getHSV(frame,dic['hsvLowHue'][0],dic['hsvHighHue'][0],
                   dic['hsvLowSaturation'][0],dic['hsvHighSaturation'][0],
                   dic['hsvLowValue'][0],dic['hsvHighValue'][0]) 
        return frame,mask
    
def applyAnalysis(frame,dic):
    if dic['analysisFlag']==['ballFlag']:
        frame,_=getThreshMedian(frame,dic['threshValue'][0],dic['medianValue'][0])
        return frame,_
    
    elif dic['filterFlag']==['cannyFilter']:
        frame,_=getCanny(frame,dic['cannyLowValue'][0],dic['cannyHighValue'][0])
        return frame,_
    
    elif dic['filterFlag']==['hsvFilter']:
        frame,mask=getHSV(frame,dic['hsvLowHue'][0],dic['hsvHighHue'][0],
                   dic['hsvLowSaturation'][0],dic['hsvHighSaturation'][0],
                   dic['hsvLowValue'][0],dic['hsvHighValue'][0]) 
        return frame,mask
    
def getProps(binaryImage):
    label_img = label(binaryImage)
    regions = regionprops(binaryImage)
    return regions
def drawProps(regions,binaryImage):
    image = cv2.cvtColor(binaryImage,cv2.COLOR_GRAY2BGR)
    for props in regions :
        y0, x0 = props.centroid
        orientation = props.orientation
        x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
        y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
        x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
        y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length
        minr, minc, maxr, maxc = props.bbox
        x0,y0,x1,y1,x2,y2 = map(int,[x0,y0,x1,y1,x2,y2])
        image = cv2.circle(image,(x0,y0),1,(0,0,255),-1)   #draw centroid
        image = cv2.line (image , (x0,y0),(x1,y1),(255,0,255),2)
        image = cv2.line (image , (x0,y0),(x2,y2),(255,0,0),2)

    return image
def wrap(binaryImage):
    #edged= cv2.Canny(binaryImage,75,200)
    
    _,contours,_= cv2.findContours(binaryImage, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    biggest_contour = max(contours, key=cv2.contourArea)

    # step 2
    hull = cv2.convexHull(biggest_contour)
    epsilon = 0.05 * cv2.arcLength(biggest_contour, True)

    # step 3
    hull = cv2.approxPolyDP(hull, epsilon, True)

    # step 4
    hull = np.float32(hull)
    method = cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT
    criteria = (method, 1000, 1e-4)
    cv2.cornerSubPix(binaryImage, hull, (5, 5), (-1, -1), criteria)
    corners = [pt[0] for pt in hull]
    if len(corners)>3:
        # Find top-right corner and use to label corners
        # Note: currently corners are in CW order
        # Note: ordering will be checked below against expected corners
        tr_index = np.argmin(c[0] + c[1] for c in corners)
        tl = corners[tr_index]
        bl = corners[(tr_index - 1) % 4]
        br = corners[(tr_index - 2) % 4]
        tr = corners[(tr_index - 3) % 4]

        # reformat and ensure that ordering is as expected below
        corners = np.float32([[c[0], c[1]] for c in [tl, bl, br, tr]])

        w = max(abs(br[0] - bl[0]),abs(tr[0] - tl[0]))  # width of paper in pixels
        h = 2 * w
        corners_pp = np.float32([[0, 0], [0, h], [w, h], [w, 0]])
        homog, mask = cv2.findHomography(corners, corners_pp)
        inv_homog, inv_mask = cv2.findHomography(corners_pp, corners)
        paper = cv2.warpPerspective(binaryImage, homog, (int(ceil(w)), int(ceil(h))))

        return paper
    else :
        return binaryImage

  
    

def applyHoughCircles(frame,dic):
    #if dic['houghCirclesFlag']==[True]:
    #try :
        circles,highlight=getHoughCircles(frame,
                                          dp=dic['houghCirclesDP'][0],
                                          minDist=dic['houghCirclesMINDIST'][0],
                                         param1=dic['houghCirclesPARAM1'][0],
                                         param2=dic['houghCirclesPARAM2'][0],
                                         minRadius=dic['houghCirclesMINRADIUS'][0],
                                         maxRadius=dic['houghCirclesMAXRADIUS'][0],
                                         blurSize=dic['houghCirclesBLURSIZE'][0])
        #dic['houghCirclesFlag']=[False]

        
        return circles,highlight
    #else : 
     #   return frame,frame
   # except : print('error');return frame,frame
    
winName='board';
board=np.zeros((950,850,3),np.uint8)              #GUI dimensions
cvui.init(winName,1);




##################################################################################################################
cap=cv2.VideoCapture(getFile());frameDic['endFrame'][0]=getVideoProp(cap)['frameCount']-1
while cv2.getWindowProperty('board', 0) >= 0 :                   #close button is not pressed
    
    #####Check load save button status
    if var['loadPickleFlag']==[True] :
        loadedPickle=readPkl(getFile());
        var=loadedPickle
        var['loadPickleFlag']=[False]
    if var['savePickleFlag']==[True] :
        writePkl('temp.pkl',var)
        var['savePickleFlag']=[False]   
########################################
    
    cv2.imshow('board',board)        #Show controllers board
    board[:]=0
    filterGroupGUI(board)
    frameSliderGUI(board,frameDic)
    statusTextGUI(board,cap)


    cv2.namedWindow('video',cv2.WINDOW_NORMAL);
    cv2.namedWindow('detect',cv2.WINDOW_NORMAL);
    
    
    frame=getFrame(frameDic['frameNumber'][0]-1,cap)
    cv2.imshow('video',frame)
    ####update the filter 
    if var["filterFlag"] ==[True]:
        cv2.namedWindow('filter',cv2.WINDOW_NORMAL);
        filterFrame,mask=applyFilter(frame,var)
        cv2.imshow('filter',filterFrame)

    ####apply hough parameters
    if var["houghCirclesFlag"] == [True] :
        cv2.namedWindow('hough',cv2.WINDOW_NORMAL);
        houghCircles,houghFrame=applyHoughCircles(mask,var)
        cv2.imshow('hough',houghFrame)
    
    
    ###Apply the line orientation
    if var["lineOrientationFlag"] ==[True]:
        
        if var["roi"]==[]: var["roi"]=[cv2.selectROI(filterFrame)]
        else : 
            r = var["roi"][0]

            crop  = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
            ret,thresh=cv2.threshold(crop,80,255,cv2.THRESH_BINARY_INV);

            cv2.namedWindow('line orientation',cv2.WINDOW_NORMAL);

            regions = getProps(thresh[:,:,0])
            drawImage = drawProps(regions,thresh[:,:,0])

            cv2.imshow('line orientation',drawImage)





    cvui.update()    
##################################################################################################################    
    

cv2.destroyAllWindows()
cap.release()



TypeError: Layout of the output array image is incompatible with cv::Mat (step[ndims-1] != elemsize or step[1] != elemsize*nchannels)

In [70]:
import math

## Analysis of parameters

In [ ]:
history=dict()
history["frame"]=[[]]*int(frameCount)
history["xyr"]=[[]]*int(frameCount)



In [ ]:
cap=cv2.VideoCapture('output.avi')
def analyzeHoughCircles(frame):
    circles = cv2.HoughCircles(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY), cv2.HOUGH_GRADIENT,  1, 20, param1 = 50, param2 = 25, minRadius=0, maxRadius=0)
    highlight=frame.copy()
    return circles


for i in range(int(frameCount)):
    _,frame=cap.read()
    if _ == False : break
    circles=analyzeHoughCircles(frame)
    
cv2.destroyAllWindows()
cap.release()   


#Choose the ball with largest radius => threshold radius?


In [ ]:
circles

def stabilize(prevCircles,curCircles,threshold):
    if curCircles is not None :
        curCircles = np.round(curCircles[0, :]).astype("int")
        prevCircles = np.round(prevCircles[0, :]).astype("int")
        if len(curCircles)==1 : return [ x,y,r for (x,y,r) in curCircles]
        elif len(curCircles)==2:
            if len(curCircles)==len(prevCircles):
                if abs(curCircles[0] - prevCircles[0]) < threshold :
                    


In [ ]:
plt.plot(list(range(int(frameCount))),[ x[0] if x!=[]  else 0 for x in history["frame"] ]);
#plt.plot(list(range(int(frameCount))),[ x[1] if x!=[]  else 0 for x in history["frame"] ]);
#plt.plot(list(range(int(frameCount))),[ x[2] if x!=[]  else 0 for x in history["frame"] ]);
#plt.legend(["x","y","r"]);
plt.xlabel('frame number');plt.ylabel('location in pixels');
plt.title('Track balls x location')
plt.grid()
plt.show()

In [ ]:
c,hh=getHoughCirlces(frame)

plt.imshow(hh)

In [ ]:
def getHoughCircles(frame):
    circles = cv2.HoughCircles(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY), cv2.HOUGH_GRADIENT,  1, 20, param1 = 50, param2 = 25, minRadius=0, maxRadius=0)
    highlight=frame.copy()
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            #highlight=cv2.circle(highlight, (x, y), r, (255, 255, 0), 4)
            highlight=cv2.rectangle(highlight, (x - 1, y - 1), (x + 1, y + 1), (255, 255, 255), -1)
    return circles,highlight

#plt.imshow(a)
#circle_img=a.copy()
#circle = cv2.HoughCircles(, cv2.HOUGH_GRADIENT, 1, 20, param1 = 50, param2 = 25, minRadius=0, maxRadius=0)

#circle = np.round(circle[0, :]).astype("int")
c,h=getHoughCircles(frame)
plt.imshow(h)
c

In [ ]:
print([i for i in dir(cv2) if i.startswith('COLOR_HSV')])

In [ ]:
import pickle
def writePkl(fileName,file):
    with open(fileName, 'wb') as output:pickle.dump(file, output, pickle.HIGHEST_PROTOCOL)
def readPkl(fileName):
    with open(fileName, 'rb') as input:return pickle.load(input)    

b=readPkl('temp.pkl')
b

In [60]:

import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.measure import regionprops

#img=cv2.imread('0.png')

#plt.imshow(img)

#frame,mask=applyFilter(img,b)
#plt.imshow(mask)

#cv2.imwrite('mask.png',mask)
mask=cv2.imread('mask.png')
def averageLine(line):
    pass
def splitLines(lines,delta):
    for i,line in enumerate(lines):
        curLine=[]
        for j in range(1,len(line)) :
            if line[j]-line[j-1]<=delta :
                curLine.append(j)

def streak(line,step):
    for i in range(1,len(line)):
        splitLine=[[]]
        if line[i]-line[i-1] <=step:splitLine[-1].append(i)
            
                
def getHorLines(binaryImage):
    height,width=binaryImage.shape
    lines=[[]]*height
    for i in range(height):lines[i]=[j for j in range(width) if binaryImage[i,j]==0]  
    return lines

def getVerLines(binaryImage):
    height,width=binaryImage.shape
    lines=[[]]*width
    for j in range(width):lines[j]=[i for i in range(height) if binaryImage[i,j]==0]  
    
    return lines




l
        



NameError: name 'l' is not defined

In [ ]:
def streak(line,step):
    splitLine=[[]]
    for i in range(1,len(line)):
        if line[i]-line[i-1] <=step:splitLine[-1].append(i)
        else :splitLine.append([i])
    return splitLine

l=[1,2,3,4,8,9,10]
a=streak(l,1)
mask=cv2.imread('mask.png',0)

def getHorLines(binaryImage):
    height,width=binaryImage.shape
    lines=[[]]*height
    for i in range(height):lines[i]=[j for j in range(width) if binaryImage[i,j]==0]  
    return lines

import statistics

def getVerLines(binaryImage):
    height,width=binaryImage.shape
    lines=[[]]*width
    for j in range(width):lines[j]=[i for i in range(height) if binaryImage[i,j]==0]  
    
    return lines

aa=[statistics.median(s) for s  in a]


hor=getHorLines(mask)
#ver=getVerLines(mask)
#centerY=[statistics.median(i) for i  in ver if i!=[]]
centerX=[statistics.median(i) for i  in hor if i!=[]]
centerX

In [ ]:
img=cv2.imread('0.png')
d=readPkl('temp.pkl')
f,mask=applyFilter(img,d)
c,_=getCanny(mask,d['cannyLowValue'][0],d['cannyHighValue'][0])
__,cc,_ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
circles = [cv2.minEnclosingCircle(cnt) for cnt in cc]
circles
out=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
for circle in circles :
    xy=(int(circle[0][0]),int(circle[0][0]))
    print(xy)
    r=int(circle[1])
    if r<50:
        out=cv2.circle(out,xy,1,(0,0,255),-1)

    
plt.imshow(out)

In [ ]:
cap_img = cv2.VideoCapture('output.avi')
import time
while (cap_img.isOpened()):
  ret, raw_img = cap_img.read()
  rgb_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)
  gray_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
  circle_img = raw_img

  #blur_img = img = cv2.medianBlur(gray_img, 5)

  circle = cv2.HoughCircles(gray_img, cv2.HOUGH_GRADIENT, 1, 20, param1 = 50, param2 = 25, minRadius=0, maxRadius=0)
  circle = np.uint16(np.around(circle))

  for i in circle[0, :]:
    cv2.circle(circle_img, (i[0], i[1]), i[2], (0, 255, 0), 2)
    cv2.circle(circle_img, (i[0], i[1]), 2, (0, 255, 255), 3)

  # canny_img = cv2.Canny(gray_img,100, 200)

  cv2.imshow('test', circle_img)


  time.sleep(0.01)

  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap_img.release()
cv2.destroyAllWindows()

In [31]:
import math
img = cv2.imread('images/4900.png')
_,threshImg=getThreshMedian(img,80,1)
#plt.imshow(threshImg[200:,400:],cmap='gray')



threshImg = threshImg[300:,:]
threshImg=cv2.bitwise_not(threshImg)
from skimage.measure import label, regionprops
highlight=img.copy()
regions=regionprops(threshImg)
fig, ax = plt.subplots()
ax.imshow(threshImg, cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    orientation = props.orientation
    x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
    y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
    x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
    y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length

    ax.plot((x0, x1), (y0, y1), '-y', linewidth=2.5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
    ax.plot(x0, y0, '.g', markersize=15)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-b', linewidth=2.5)

#ax.axis((0, 600, 600, 0))
plt.show()

In [4]:
import matplotlib.pyplot as plt

from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
import skimage

# Load picture, convert to grayscale and detect edges
image_rgb = skimage.io.imread('images/7420.png')
#plt.imshow(image_rgb)

image_gray = color.rgb2gray(image_rgb)
edges = canny(image_gray, sigma=2.0,
              low_threshold=0.55, high_threshold=0.8)

# Perform a Hough Transform
# The accuracy corresponds to the bin size of a major axis.
# The value is chosen in order to get a single high accumulator.
# The threshold eliminates low accumulators
result = hough_ellipse(edges, accuracy=20, threshold=250,
                       min_size=100, max_size=120)
result.sort(order='accumulator')

# Estimated parameters for the ellipse
best = list(result[-1])
yc, xc, a, b = [int(round(x)) for x in best[1:5]]
orientation = best[5]

# Draw the ellipse on the original image
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
image_rgb[cy, cx] = (0, 0, 255)
# Draw the edge (white) and the resulting ellipse (red)
edges = color.gray2rgb(img_as_ubyte(edges))
edges[cy, cx] = (250, 0, 0)

fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4),
                                sharex=True, sharey=True)

ax1.set_title('Original picture')
ax1.imshow(image_rgb)

ax2.set_title('Edge (white) and result (red)')
ax2.imshow(edges)

plt.show()


KeyboardInterrupt: 

In [33]:
%%latex
\begin{aligned} M_{ij}  = \sum\limits_{y=0}^{nrows}\sum\limits_{x=0}^{ncols} x^i \ y^j \ I(x, y) \end{aligned}

<IPython.core.display.Latex object>

In [8]:
import cv2
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib  qt

#3233
fileName = "images/5700.png"   #enter file name 
img = cv2.imread(fileName,0)
ret,img=cv2.threshold(img,80,255,cv2.THRESH_BINARY_INV);
img =img [350:-20,:]   #optimize 



# img=img[350:,:]

def raw_moment(data, i_order, j_order):
  nrows, ncols = data.shape
  y_indices, x_indicies = np.mgrid[:nrows, :ncols]
  return (data * x_indicies**i_order * y_indices**j_order).sum()


def moments_cov(data):
  data_sum = data.sum()
  m10 = raw_moment(data, 1, 0)   #M10
  m01 = raw_moment(data, 0, 1)   #M01
  x_centroid = m10 / data_sum
  y_centroid = m01 / data_sum
  u11 = (raw_moment(data, 1, 1) - x_centroid * m01) / data_sum
  u20 = (raw_moment(data, 2, 0) - x_centroid * m10) / data_sum
  u02 = (raw_moment(data, 0, 2) - y_centroid * m01) / data_sum
  cov = np.array([[u20, u11], [u11, u02]])
  return cov


y, x = np.nonzero(img)
x = x - np.mean(x)
y = y - np.mean(y)
coords = np.vstack([x, y])


cov = moments_cov(img)
evals, evecs = np.linalg.eig(cov)

plt.figure(2)

sort_indices = np.argsort(evals)[::-1]
x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
x_v2, y_v2 = evecs[:, sort_indices[1]]


scale = 20
plt.plot([x_v1*-scale*2, x_v1*scale*2],
         [y_v1*-scale*2, y_v1*scale*2], color='black')
plt.plot([x_v2*-scale, x_v2*scale],
         [y_v2*-scale, y_v2*scale], color='blue')

plt.plot(x, y, 'r.')
plt.axis('equal')
plt.gca().invert_yaxis()  # Match the image system with origin at top left
plt.show()



# theta = np.arctan((x_v1)/(y_v1))  
# rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
#                       [np.sin(theta), np.cos(theta)]])
# transformed_mat = rotation_mat * coords


# x_transformed, y_transformed = transformed_mat.A
# plt.plot(x_transformed, y_transformed, 'gx')

# cv2.imshow('original',img)
# key = cv2.waitKey(0) & 0xff
plt.figure(3);plt.imshow(img)
